In [1]:
# Note: This is a hack to allow importing from the parent directory
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))

In [2]:
import torch
from configs import Config
from torchvision import datasets
from models import GaussianImageTrainer
from constants import CIFAR10_TRANSFORM
from data import CIFAR10GaussianSplatsDataset
from utils import (
    generate_random_splat,
    merge_spherical_harmonics,
    save_gs_data,
    load_gs_data,
    tensor_to_image,
)

In [3]:
NUM_POINTS = 1024  # 32x32
CIFAR10 = datasets.CIFAR10(
    root="../data/CIFAR10/train",
    train=True,
    download=True,
    transform=CIFAR10_TRANSFORM,
)
means, quats, scales, opacities, colors, viewmats, Ks, sh0, shN = generate_random_splat(
    NUM_POINTS
)
# image = tensor_to_image(CIFAR10[0][0])
image = CIFAR10[0][0]
label = CIFAR10[0][1]
splat = torch.nn.ParameterDict(
    {
        "mean": torch.nn.Parameter(means),
        "quat": torch.nn.Parameter(quats),
        "scale": torch.nn.Parameter(scales),
        "opacity": torch.nn.Parameter(opacities),
        "color": torch.nn.Parameter(colors),
        "viewmat": torch.nn.Parameter(viewmats),
        "Ks": torch.nn.Parameter(Ks),
        "sh0": torch.nn.Parameter(sh0),
        "shN": torch.nn.Parameter(shN),
    }
)
print(image, label, splat)

splat = merge_spherical_harmonics(splat)
save_gs_data(image, label, splat, Path("../data/CIFAR10GS/knn/train/test.pt"))
image, label, splat = load_gs_data(Path("../data/CIFAR10GS/knn/train/test.pt"))
print(image, label, splat)

Files already downloaded and verified
tensor([[[0.2314, 0.2431, 0.2471],
         [0.1686, 0.1804, 0.1765],
         [0.1961, 0.1882, 0.1686],
         ...,
         [0.6196, 0.5176, 0.4235],
         [0.5961, 0.4902, 0.4000],
         [0.5804, 0.4863, 0.4039]],

        [[0.0627, 0.0784, 0.0784],
         [0.0000, 0.0000, 0.0000],
         [0.0706, 0.0314, 0.0000],
         ...,
         [0.4824, 0.3451, 0.2157],
         [0.4667, 0.3255, 0.1961],
         [0.4784, 0.3412, 0.2235]],

        [[0.0980, 0.0941, 0.0824],
         [0.0627, 0.0275, 0.0000],
         [0.1922, 0.1059, 0.0314],
         ...,
         [0.4627, 0.3294, 0.1961],
         [0.4706, 0.3294, 0.1961],
         [0.4275, 0.2863, 0.1647]],

        ...,

        [[0.8157, 0.6667, 0.3765],
         [0.7882, 0.6000, 0.1333],
         [0.7765, 0.6314, 0.1020],
         ...,
         [0.6275, 0.5216, 0.2745],
         [0.2196, 0.1216, 0.0275],
         [0.2078, 0.1333, 0.0784]],

        [[0.7059, 0.5451, 0.3765],
         

In [4]:
# Example of how to use the CIFAR10GaussianSplatsDataset
dataset = CIFAR10GaussianSplatsDataset(
    root="../data/CIFAR10GS",
    train=True,
    init_type="knn",
)
dataset[0]

(<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
 6,
 ParameterDict(
     (Ks): Parameter containing: [torch.FloatTensor of size 3x3]
     (color): Parameter containing: [torch.FloatTensor of size 1024x3]
     (mean): Parameter containing: [torch.FloatTensor of size 1024x3]
     (opacity): Parameter containing: [torch.FloatTensor of size 1024x1]
     (quat): Parameter containing: [torch.FloatTensor of size 1024x4]
     (scale): Parameter containing: [torch.FloatTensor of size 1024x3]
     (viewmat): Parameter containing: [torch.FloatTensor of size 4x4]
 ))

In [5]:
# Example on how the CIFAR10GS dataset should be created
trainer = GaussianImageTrainer(Config())

Model initialized. Number of Gaussians: 1024


/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(
/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(


In [6]:
start_idx = 622  # Final trained image
end_idx = 5000 # Train only the first 5000 images

In [9]:
for idx in range(start_idx, end_idx):
    image, label = CIFAR10[idx]
    
    # Initialize trainer with current image
    trainer.reinitialize(Config(image=image))
    
    # Train and process splat
    splat = trainer.train()
    splat = merge_spherical_harmonics(splat)
    
    # Save with original dataset index
    save_gs_data(
        image,
        label,
        splat,
        Path(f".../{'train' if CIFAR10.train else 'test'}/{idx}.pt")
    )

Model initialized. Number of Gaussians: 1024


/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(
/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(


Loss: 0.189 (L1: 0.013, L2: 0.000, SSIM: 0.090): 100%|████████| 1000/1000 [00:12<00:00, 80.55step/s]


Final loss: 0.18924503028392792
Total Time: Rasterization: 2.845s, Backward: 5.240s
Model initialized. Number of Gaussians: 1024


Loss: 0.203 (L1: 0.035, L2: 0.004, SSIM: 0.109): 100%|████████| 1000/1000 [00:12<00:00, 83.03step/s]


...

Final loss: 0.19385850429534912
Total Time: Rasterization: 2.779s, Backward: 5.070s
Model initialized. Number of Gaussians: 1024


Loss: 0.148 (L1: 0.015, L2: 0.001, SSIM: 0.040): 100%|████████| 1000/1000 [00:12<00:00, 82.96step/s]


Final loss: 0.1474795639514923
Total Time: Rasterization: 2.807s, Backward: 5.124s
Model initialized. Number of Gaussians: 1024


Loss: 0.230 (L1: 0.014, L2: 0.001, SSIM: 0.042):  58%|█████▏   | 583/1000 [00:06<00:04, 91.40step/s]


KeyboardInterrupt: 